In [ ]:
oneailib_version = 1.13
oneailib_to_install = f"oneailib=={oneailib_version}" if oneailib_version else "oneailib"

In [ ]:
!pip install $oneailib_to_install --extra-index-url https://artifactory.us.caas.oneadp.com/artifactory/api/pypi/datacloud-datascience-pypi-local/simple

In [ ]:
dbutils.library.restartPython()

In [ ]:
from oneailib.embeddings.azure_openai import AzureOpenAIEmbedding
from oneailib.chat_models.azure_openai import AzureOpenAIChatModel
from oneailib.document_loaders.sql import SQLLoader
from oneailib.core.chat_models.base import FunctionCalling, FunctionCallingParameters, FunctionCallingPropertiesDocumentKey
from oneailib.core.documents.base import Document
from oneailib.document_transformers.transformers import EmbedKeyTransformer
from oneailib.prompt.openai import OpenAIPromptTemplate
from oneailib.document_transformers.metadata_transformers import (
    DecomposeKeysTransformer,
    AddDocumentIDTransformer, 
    AddDocumentVersionTransformer,
    AddMetadataTransformer,
    DocumentInclusionConditionsMetadataTransformer,
)

import os

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

print("✓ Imports successful")

In [ ]:
dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
dbutils.widgets.text("volume", "datacloud-dita-gen-volume/FAQ_update/", "Volume")

In [ ]:
CATALOG_NAME = dbutils.widgets.get("catalog")
SCHEMA_NAME = dbutils.widgets.get("schema")
VOLUME_NAME = dbutils.widgets.get("volume")

In [ ]:
# os.environ["SERVICE_ACCOUNT_NAME"] = "service-cred-datacloud-dmt-jc-dit"

scope_name = "mishrapr"
os.environ["AZURE_TENANT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_TENANT_ID")
os.environ["AZURE_CLIENT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_ID")
os.environ["AZURE_CLIENT_SECRET"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_SECRET")

# scope_name = "FAQ_API"
# os.environ["AZURE_TENANT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_TENANT_ID")
# os.environ["AZURE_CLIENT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_ID")
# os.environ["AZURE_CLIENT_SECRET"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_SECRET")

In [ ]:
# ============================================================================
# REPLACED: Now using faq_generation module (moved to faq_generation/prompts.py and faq_generation/schemas.py)
# ============================================================================
# This cell has been replaced by importing from the faq_generation module
# See: faq_update/faq_generation/prompts.py and faq_update/faq_generation/schemas.py

from faq_generation.prompts import FAQ_QUESTION_GENERATION_PROMPT
from faq_generation.schemas import FAQ_QUESTION_GENERATION_SCHEMA

# The prompt and schema are now imported rather than defined inline
# This makes the code more maintainable and reusable
faq_prompt_template = FAQ_QUESTION_GENERATION_PROMPT
faq_function_calling = FAQ_QUESTION_GENERATION_SCHEMA

print("✓ FAQ generation prompts and schemas imported from faq_generation module")

In [ ]:
# Configuration constants
text_chunk_field = "chunk_text"
question_field = "question"
embedding_model_name = "text-embedding-ada-002_2-pgo-amrs"
chat_model_name = "gpt-4o_2024-11-20-pgo-amrs"

In [ ]:
# Configuration
import os

# Chunking configuraation
CHUNK_SIZE = 1000  # Characters per chunk

# Check if we're running in Databricks
def is_databricks_environment():
    """Check if we're running in a Databricks environment."""
    return 'DATABRICKS_RUNTIME_VERSION' in os.environ

if is_databricks_environment():
    print("🔧 Databricks environment detected!")
    
    # Unity Catalog paths
    PROJECT_ROOT = Path(f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}")
    DB_PATH = None  # Unity Catalog doesn't use file paths for databases
    DATA_DIR = PROJECT_ROOT / "data_ingestion" / "data"
    MARKDOWN_DIR = DATA_DIR / "markdown"
    USE_UNITY_CATALOG = True
    
    print(f"Unity Catalog Configuration:")
    print(f"  Catalog: {CATALOG_NAME}")
    print(f"  Schema: {SCHEMA_NAME}")
    print(f"  Volume: {VOLUME_NAME}")
    print(f"  Project Root: {PROJECT_ROOT}")
    print(f"  Data Directory: {DATA_DIR}")
    print(f"  Markdown Directory: {MARKDOWN_DIR}")
    print(f"  Database: Unity Catalog managed tables")
    
else:
    print("💻 Local environment detected")
    
    # Original local configuration
    PROJECT_ROOT = Path.cwd().parent if 'granular_impact' in str(Path.cwd()) else Path.cwd()
    DB_PATH = PROJECT_ROOT / "databases" / "my_database.db"
    DATA_DIR = Path("data_ingestion/data")
    MARKDOWN_DIR = DATA_DIR / "markdown"
    USE_UNITY_CATALOG = False
    
    print(f"Local File System Configuration:")
    print(f"  Project Root: {PROJECT_ROOT}")
    print(f"  Database: {DB_PATH}")
    print(f"  Data Directory: {DATA_DIR}")
    print(f"  Markdown Directory: {MARKDOWN_DIR}")

# Path verification
if USE_UNITY_CATALOG:
    # For Databricks, create directories if they don't exist
    try:
        DATA_DIR.mkdir(parents=True, exist_ok=True)
        MARKDOWN_DIR.mkdir(parents=True, exist_ok=True)
        print("\n✓ Unity Catalog directories created/verified")
    except Exception as e:
        print(f"\n❌ Error creating directories: {e}")
        print(f"Make sure you have created the Unity Catalog volume:")
        print(f"CREATE VOLUME IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME};")
        
else:
    # Original verification for local environment
    assert DB_PATH.exists(), f"Database not found: {DB_PATH}"
    assert DATA_DIR.exists(), f"Data directory not found: {DATA_DIR}"
    assert MARKDOWN_DIR.exists(), f"Markdown directory not found: {MARKDOWN_DIR}"
    print("\n✓ All paths verified")

In [ ]:
from oneailib.document_loaders.sql import SQLLoader

# Define your configuration parameters
path_to_input_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.content_chunks"
content_md_field = "chunk_text"  # Replace with actual content column name

# Define the fields schema
fields_schema = {
    "chunk_id": "",
    "chunk_text": "",
    "ud_source_file_id": "",
    "chunk_index": "",
    "content_checksum": "",
    "created_at": "",
    "status": "",
}

# Create the SQL query
query = f"""
SELECT *
FROM {path_to_input_table}
"""

# Initialize the SQLLoader
sql_loader = SQLLoader(
    query=query,
    content_column=content_md_field,
    fields_schema=fields_schema
)

# Load the documents
chunk_documents = sql_loader.load()

# Display results
print(f"Loaded {len(chunk_documents)} documents")
for i, doc in enumerate(chunk_documents[:3]):  # Show first 3 documents
    print(f"\nDocument {i+1}:")
    print(f"Content preview: {doc.page_content[:200]}...")
    print(f"Metadata: {doc.metadata}")

In [ ]:
# ============================================================================
# OPTION 1: Using the NEW faq_generation module (RECOMMENDED)
# ============================================================================

from faq_generation import QuestionGenerator

# Create question generator
question_generator = QuestionGenerator(
    chat_model_name=chat_model_name,
    temperature=0.0,
    max_tokens=4000,
    max_retries=10,
    seconds_between_retries=5,
    max_questions=5  # Configure max questions per chunk
)

# Generate questions from documents
print(f"Generating questions from {len(chunk_documents)} documents...")
question_generated_docs = question_generator.generate_from_documents(
    documents=chunk_documents,
    add_ids=True,
    add_versions=True
)

print(f"✓ Generated {len(question_generated_docs)} questions")

# ============================================================================
# OPTION 2: Original approach (kept for backward compatibility)
# ============================================================================
# Uncomment below if you want to use the original approach:

# question_generator_model = AzureOpenAIChatModel(
#     engine_name=chat_model_name,
#     temperature=0.0,
#     max_tokens=4000,
#     max_retries=10,
#     seconds_between_retries=5,
#     prompt=faq_prompt_template,
#     function_calling=faq_function_calling,
# )
#
# question_decomposer = DecomposeKeysTransformer(
#     keys_to_decompose={"questions_for_single_cluster": ["question"]}
# )
#
# question_generated_docs = question_generator_model(chunk_documents)

# ============================================================================

In [ ]:
# ============================================================================
# NOTE: Decomposition is now handled internally by QuestionGenerator
# ============================================================================
# The new faq_generation module handles decomposition internally in the
# generate_from_documents() method, so we can skip this step.
# The question_generated_docs already contains decomposed questions.

# If you're using the original approach (Option 2 above), uncomment this:
# decomposed_question_docs = question_decomposer.transform_documents(question_generated_docs)

# For the new approach, just use the already-decomposed documents:
decomposed_question_docs = question_generated_docs

print(f"✓ Using {len(decomposed_question_docs)} decomposed question documents")

In [ ]:
# ============================================================================
# NOTE: ID transformation is now handled internally by QuestionGenerator
# ============================================================================
# The new faq_generation module adds document IDs internally when you set
# add_ids=True in generate_from_documents(), so we can skip this step.

# If you're using the original approach (Option 2 above), uncomment this:
# id_transformer = AddDocumentIDTransformer()
# documents_with_ids = id_transformer.transform_documents(decomposed_question_docs)

# For the new approach, IDs are already added:
documents_with_ids = decomposed_question_docs

print(f"✓ Using {len(documents_with_ids)} documents with IDs")

In [ ]:
# ============================================================================
# NOTE: Version transformation is now handled internally by QuestionGenerator
# ============================================================================
# The new faq_generation module adds document versions internally when you set
# add_versions=True in generate_from_documents(), so we can skip this step.

# If you're using the original approach (Option 2 above), uncomment this:
# version_transformer = AddDocumentVersionTransformer()
# documents_with_version = version_transformer.transform_documents(documents_with_ids)

# For the new approach, versions are already added:
documents_with_version = documents_with_ids

print(f"✓ Using {len(documents_with_version)} documents with versions")

In [ ]:
# Convert documents to Spark DataFrame
df = spark.createDataFrame([doc.metadata for doc in documents_with_version])

In [ ]:
question_data = []
for i, q in df.toPandas().iterrows():
    question_data.append({
        'question_id': q.id,
        'question_text': q.question,
        'source_type': 'document',
        'generation_method': "LLM",
        "status": q.status
    })
df_questions = pd.DataFrame(question_data)
sdf_questions = spark.createDataFrame(df_questions)

In [ ]:
# Map to faq_question_sources table
sources_data = []
for i, q in df.toPandas().iterrows():
    if q.content_checksum:
        sources_data.append({
            'question_id': q.id,
            'content_checksum': q.content_checksum,
            # 'chunk_version': q.version,quest
            # 'chunk_content': q.chunk_text,
            'is_primary_source': True,
            'contribution_weight': 1.0,
            'is_valid': True,
        })

df_question_sources = pd.DataFrame(sources_data)
sdf_question_sources = spark.createDataFrame(df_question_sources)

In [ ]:
sdf_questions.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.faq_questions")

In [ ]:
sdf_question_sources.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.faq_question_sources")

#---------------------------------------------------

In [ ]:
# 2. Generate the embeddings for text_content, html_content Azure OpenAI embedding model
embedding_model = AzureOpenAIEmbedding(
    engine_name=embedding_model_name,
)

text_chunk_embedding_transformer = EmbedKeyTransformer(
    key=text_chunk_field,
    embeddings=embedding_model
)

question_embedding_transformer = EmbedKeyTransformer(
    key=question_field,
    embeddings=embedding_model
)


question_embedded_docs = question_embedding_transformer.transform_documents(decomposed_question_docs)
text_chunk_embedded_docs = text_chunk_embedding_transformer.transform_documents(chunk_documents)